# Unsupervised Learning: Clustering Lab





In [3]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn import metrics
from sklearn import preprocessing
from scipy.io import arff
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1. (50%) Implement the k-means clustering algorithm and the HAC (Hierarchical Agglomerative Clustering) algorithm.

### 1.1.1 HAC

### Code requirements 
- HAC should support both single link and complete link options.
- HAC automatically generates all clusterings from n to 1.  To simplify the amount of output you may want to implement a mechanism to specify for which k values actual output will be generated.


---
The output should include the following:
- The number of clusters (k).
- The silhouette score of the full clustering. (You can either write and use your own silhouette_score function (extra credit) or use sklearn's)


For each cluster report include:


- The centroid id.
- The number of instances tied to that centroid. 
---

In [35]:
class HACClustering(BaseEstimator,ClassifierMixin):

    def __init__(self,k=3,link_type='single'): ## add parameters here
        """
        Args:
            k = how many final clusters to have
            link_type = single or complete. when combining two clusters use complete link or single link
        """
        self.link_type = link_type
        self.k = k
        self.clusters = []
        
    def fit(self, X, y=None):
        """ Fit the data; In this lab this will make the K clusters :D
        Args:
            X (array-like): A 2D numpy array with the training data
            y (array-like): An optional argument. Clustering is usually unsupervised so you don't need labels
        Returns:
            self: this allows this to be chained, e.g. model.fit(X,y).predict(X_test)
        """
        cluster_arr = [np.asarray([item]) for item in X]
        self.clusters = cluster_arr
        is_done = False
        while not is_done:
          # print('clusters: ', len(self.clusters))
          if len(self.clusters) == self.k:
            is_done = True
            break
          if self.link_type == 'single':
            self.clusters = self.single_cluster(self.clusters)
          else:
            self.clusters = self.cluster_complete_link(self.clusters)
        return self

    def score(self):
      self.sse = []
      self.total_sse = 0
      self.centroids = []
      # find centroids
      for i in range(len(self.clusters)):
        centroid = np.mean(self.clusters[i], axis=0)
        self.centroids.append(centroid)
        # find cluster sse
        tmp = np.linalg.norm(self.clusters[i] - centroid)
        squared_err = tmp**2
        self.sse.append(np.sum(squared_err))
        self.total_sse += squared_err

    def single_cluster(self, clusters):
      distances = []
      cluster_indexes = []
      for i in range(len(clusters)):
        tmp_dist = []
        for j in range(len(clusters)):
          min_dist = np.inf
          if i == j:
            tmp_dist.append(np.inf)
            continue
          if len(clusters[i]) > 1:
            # check if j is also a cluster
            if len(clusters[j]) > 1:
            # calc distance between the two clusters
              for k in range(len(clusters[i])):
                for l in range(len(clusters[j])):
                  dist = np.linalg.norm(clusters[i][k] - clusters[j][l])
                  if dist < min_dist:
                    min_dist = dist
            else: 
              for k in range(len(clusters[i])):
                dist = np.linalg.norm(clusters[i][k] - clusters[j])
                if dist < min_dist:
                  min_dist = dist
          else: 
            if len(clusters[j]) > 1:
            # i is single cluster, j has more than one
              for k in range(len(clusters[j])):
                dist = np.linalg.norm(clusters[i] - clusters[j][k])
                if dist < min_dist:
                  min_dist = dist
            else:
              min_dist = np.linalg.norm(clusters[i] - clusters[j])
          tmp_dist.append(min_dist)
        min_val = min(tmp_dist)
        index = tmp_dist.index(min_val)
        distances.append(min_val)
        cluster_indexes.append(index)
      
      best_dist = min(distances)
      dist_index = distances.index(best_dist)
      prev_index = cluster_indexes[dist_index]
      
      comb_arr = clusters[prev_index]
      comb_arr = np.append(clusters[dist_index], np.array(clusters[prev_index]), axis=0)
      clusters[dist_index] = comb_arr
      new_clusters = np.delete(clusters, prev_index)
      # clusters[prev_index].pop()
      return new_clusters

    def cluster_complete_link(self, clusters):
      distances = []
      cluster_indexes = []
      for i in range(len(clusters)):
        tmp_dist = []
        for j in range(len(clusters)):
          min_dist = 0
          if i == j:
            tmp_dist.append(np.inf)
            continue
          if len(clusters[i]) > 1:
            # check if j is also a cluster
            if len(clusters[j]) > 1:
            # calc distance between the two clusters
              for k in range(len(clusters[i])):
                for l in range(len(clusters[j])):
                  dist = np.linalg.norm(clusters[i][k] - clusters[j][l]) 
                  if dist > min_dist:
                    min_dist = dist
            else: 
              for k in range(len(clusters[i])):
                dist = np.linalg.norm(clusters[i][k] - clusters[j])
                if dist > min_dist:
                  min_dist = dist
          else: 
            if len(clusters[j]) > 1:
            # i is single cluster, j has more than one
              for k in range(len(clusters[j])):
                dist = np.linalg.norm(clusters[i] - clusters[j][k])
                if dist > min_dist:
                  min_dist = dist
            else:
              min_dist = np.linalg.norm(clusters[i] - clusters[j])
          tmp_dist.append(min_dist)
        min_val = min(tmp_dist)
        index = tmp_dist.index(min(tmp_dist))
        distances.append(min_val)
        cluster_indexes.append(index)
      
      best_dist = min(distances)
      dist_index = distances.index(best_dist)
      prev_index = cluster_indexes[dist_index]
      
      comb_arr = clusters[prev_index]
      comb_arr = np.append(clusters[dist_index], np.array(clusters[prev_index]), axis=0)
      clusters[dist_index] = comb_arr
      new_clusters = np.delete(clusters, prev_index)
      # clusters[prev_index].pop()
      return new_clusters
    
    def print_clusters(self):
      """
        Used for grading.
        print("Num clusters: {:d}\n".format(k))
        print("Silhouette score: {:.4f}\n\n".format(silhouette_score))
        for each cluster and centroid:
          print(np.array2string(centroid,precision=4,separator=","))
          print("{:d}\n".format(size of cluster))
      """
      # print results
      print("Num clusters: {:d}".format(len(self.clusters)))
      print("SSE score: {:.4f}\n\n".format(self.total_sse))
      for i in range(len(self.clusters)):
        print(np.array2string(self.centroids[i],precision=4,separator=","))
        print("{:d}".format(len(self.clusters[i])))
        print("{:.4f}\n".format(self.sse[i]))
      pass
    

### 1.1.2 Debug 

Debug your model by running it on the [Debug Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/abalone.arff)


---
The dataset was modified to be a lot smaller. The last datapoint should be on line 359 or the point 0.585,0.46,0.185,0.922,0.3635,0.213,0.285,10. The remaining points should be commented out.


- Make sure to include the output class (last column) as an additional input feature
- Normalize Data
- K = 5
- Use 4 decimal places and DO NOT ROUND when reporting silhouette score and centroid values.


---
Solutions in files:

[Debug HAC Single (Silhouette).txt](https://raw.githubusercontent.com/cs472ta/CS472/master/debug_solutions/Debug%20HAC%20Single%20Link%20%28Silhouette%29.txt)

[Debug HAC Complete (Silhouette).txt](https://raw.githubusercontent.com/cs472ta/CS472/master/debug_solutions/Debug%20HAC%20Complete%20Link%20%28Silhouette%29.txt)

In [5]:
def normalize_data(inputs):
  xmin = inputs.min(axis=0)
  xmax = inputs.max(axis=0)
  return (inputs-xmin)/(xmax-xmin)

In [36]:
# Debug Here
!curl -s https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/abalone.arff --output debug.arff
# Train on training set
debug_data = arff.loadarff('debug.arff')
debug_np = np.array(debug_data[0])
debug_norm = np.array(normalize_data(pd.DataFrame(debug_np)))

# print('SINGLE LINK')
# clf = HACClustering(k=5, link_type='single')
# res = clf.fit(debug_norm)
# res.score()

print('COMPLETE LINK')
clf = HACClustering(k=5, link_type='complete')
res = clf.fit(debug_norm)
res.score()
res.print_clusters()

COMPLETE LINK
Num clusters: 5
SSE score: 13.0824


[0.6544,0.649 ,0.5256,0.2879,0.2815,0.3057,0.2288,0.3911]
71
3.8232

[0.3661,0.3505,0.271 ,0.1008,0.1024,0.1058,0.0836,0.2116]
67
5.2786

[0.7622,0.7658,0.6759,0.4265,0.4016,0.4536,0.3376,0.5217]
38
1.4989

[0.8818,0.8904,0.7582,0.614 ,0.5433,0.5317,0.561 ,0.7794]
16
1.5328

[0.9471,0.934 ,0.8158,0.7457,0.6434,0.7944,0.6457,0.625 ]
8
0.9490



### 1.1.3 Evaluation

We will evaluate your model based on its print_clusters() output using [Evaluation Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/seismic-bumps_train.arff)

- Make sure to include the output class (last column) as an additional input feature
- Normalize Data
- K = 5
- Use 4 decimal places and DO NOT ROUND when reporting silhouette score and centroid values.

#### 1.1.3.1 Complete Link

#### 1.1.3.1 Single Link

In [ ]:
# Load evaluation data
!curl -s https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/seismic-bumps_train.arff --output evaldata.arff
eval_data = arff.loadarff('evaldata.arff')
eval_np = np.array(eval_data[0])
eval_norm = np.array(normalize_data(pd.DataFrame(eval_data)))

# Train on evaluation data using complete link
clf = HACClustering(k=5, link_type='complete')
res = clf.fit(debug_norm)
res.score()

# Print clusters
res.print_clusters()


In [11]:
# Train on evaluation data using single link
clf = HACClustering(k=5, link_type='single')
res = clf.fit(eval_norm)

# Print clusters
res.score()
res.print_clusters()

c:\Users\Alex\OneDrive\Desktop\Fall2021\CS472\CS472-Labs\.venv\lib\site-packages\numpy\lib\function_base.py:4454: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)


5
0.10710164554014481 

[0.07330122 0.07919399 0.08424397 0.07975487 0.07883174 0.0627566
 0.07713756 0.05345225]
67
0.04507390918155194 

[0.0761997  0.07961572 0.08675442 0.07806886 0.08295861 0.13452285
 0.07511014 0.05345225]
2
0.0001502428532181635 

[0.07310257 0.07850764 0.08570988 0.0780761  0.08026918 0.166119
 0.07575392 0.05345225]
1
0.0 

[0.0937976  0.08936042 0.0846827  0.08891764 0.08948263 0.0894777
 0.09119145 0.1069045 ]
69
0.06187749350537471 

[0.09798201 0.09202625 0.08358246 0.09054628 0.09095389 0.16604446
 0.09168935 0.1069045 ]
1
0.0 



### 1.2.1 K-Means

### Code requirements 
- Ability to choose k and specify k initial centroids
- Use Euclidean Distance as metric
- Ability to handle distance ties
- Include output label as a cluster feature


---
The output should include the following:
- The number of clusters (k).
- The silhouette score of the full clustering. (You can either write and use your own silhouette_score function (extra credit) or use sklearn's)


For each cluster report include:


- The centroid id.
- The number of instances tied to that centroid. 
---
You only need to handle continuous features

In [41]:
class KMEANSClustering(BaseEstimator,ClassifierMixin):

    def __init__(self,k=3,debug=False): ## add parameters here
        """
        Args:
            k = how many final clusters to have
            debug = if debug is true use the first k instances as the initial centroids otherwise choose random points as the initial centroids.
        """
        self.k = k
        self.debug = debug

    def fit(self, X, y=None):
        """ Fit the data; In this lab this will make the K clusters :D
        Args:
            X (array-like): A 2D numpy array with the training data
            y (array-like): An optional argument. Clustering is usually unsupervised so you don't need labels
        Returns:
            self: this allows this to be chained, e.g. model.fit(X,y).predict(X_test)
        """
        self.X = X
        self.centroids = X[:self.k]
        self.clusters = [np.array([]) for k in range(self.k)]
        is_done = False
        while not is_done:
            if not self.update_clusters(X):
                is_done = True
                break
        return self

    def update_centroids(self):
        new_centroids = []
        updated = False
        for i, cluster in enumerate(self.clusters):
            centroid = np.mean(cluster, axis=0)
            if not np.array_equal(self.centroids[i], centroid):
                updated = True
            new_centroids.append(centroid)
        self.centroids = new_centroids
        return updated

    def update_clusters(self, points):
        self.clusters = [np.array([]) for k in range(self.k)]
        # for each point see which centroid is closest
        for i in range(len(points)):
            # get shortest distance between centroids
            min_val = np.inf
            ind = 0
            for j in range(len(self.centroids)):
                dist = np.linalg.norm(points[i] - self.centroids[j])
                if dist < min_val:
                    min_val = dist
                    ind = j
            if len(self.clusters[ind]) == 0:
                add = np.array([points[i]])
                self.clusters[ind] = add
            else:
                original = self.clusters[ind]
                original = np.append(original, np.array([points[i]]), axis=0)
                self.clusters[ind] = original
        
        return self.update_centroids()
    
    def score(self):
        self.sse = []
        self.total_sse = 0
        # find centroids
        for i in range(len(self.clusters)):
            # find cluster sse
            tmp = np.linalg.norm(self.clusters[i] - self.centroids[i])
            squared_err = tmp**2
            self.sse.append(np.sum(squared_err))
            self.total_sse += squared_err
            
    
    def print_clusters(self):
        """
            Used for grading.
            print("Num clusters: {:d}\n".format(k))
            print("Silhouette score: {:.4f}\n\n".format(silhouette_score))
            for each cluster and centroid:
                print(np.array2string(centroid,precision=4,separator=","))
                print("{:d}\n".format(size of cluster))
        """
        print("{:d}".format(self.k))
        print("{:.4f}\n".format(self.total_sse))
        for i in range(len(self.clusters)):
            print(np.array2string(self.centroids[i],precision=4,separator=","))
            print("{:d}".format(len(self.clusters[i])))
            print("{:.4f}\n".format(self.sse[i]))


### 1.2.2 Debug 

Debug your model by running it on the [Debug Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/abalone.arff)


- Train until convergence
- Make sure to include the output class (last column) as an additional input feature
- Normalize Data
- K = 5
- Use the first k instances as the initial centroids
- Use 4 decimal places and DO NOT ROUND when reporting silhouette score and centroid values




---
Solutions in files:

[Debug K Means (Silhouette).txt](https://raw.githubusercontent.com/cs472ta/CS472/master/debug_solutions/Debug%20K%20Means%20%28Silhouette%29.txt)

In [42]:
# Load debug data
clf = KMEANSClustering(k=5)

# Train on debug data
res = clf.fit(debug_norm)
res.score()

# Print clusters
res.print_clusters()

5
9.7826

[0.7325,0.7327,0.627 ,0.3817,0.3633,0.4045,0.3046,0.4839]
75
4.0454

[0.3704,0.3519,0.2686,0.0926,0.0935,0.094 ,0.0792,0.218 ]
34
0.6609

[0.9035,0.905 ,0.7774,0.6579,0.5767,0.6193,0.5893,0.7279]
24
3.2116

[0.5692,0.5628,0.4376,0.211 ,0.2113,0.2248,0.1659,0.317 ]
54
1.5452

[0.1296,0.1037,0.1053,0.0177,0.0211,0.0272,0.0135,0.0724]
13
0.3195



### 1.2.3 Evaluation

We will evaluate your model based on its print_clusters() output using [Evaluation Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/seismic-bumps_train.arff)
- Train until convergence
- Make sure to include the output class (last column) as an additional input feature
- Normalize Data
- K = 5
- Use the first k instances as the initial centroids
- Use 4 decimal places and DO NOT ROUND when reporting silhouette score and centroid values

In [47]:
# Load evaluation data
!curl -s https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/seismic-bumps_train.arff --output k_evaldata.arff
k_eval_data = arff.loadarff('k_evaldata.arff')
k_eval_np = np.array(k_eval_data[0])
k_eval_norm = preprocessing.normalize(pd.DataFrame(k_eval_np), axis=0)
clf = KMEANSClustering(k=5)

# Train on evaluation data
res = clf.fit(k_eval_norm)
res.score()

# Print clusters
res.print_clusters()

5
0.0477

[0.0926,0.0888,0.0846,0.0886,0.0889,0.083 ,0.091 ,0.1069]
32
0.0068

[0.0738,0.0794,0.0844,0.0798,0.0792,0.0439,0.0771,0.0535]
35
0.0093

[0.0925,0.0887,0.0846,0.0881,0.089 ,0.1257,0.0898,0.1028]
26
0.0186

[0.0727,0.0789,0.0842,0.0796,0.0785,0.0848,0.077 ,0.0535]
33
0.0118

[0.0968,0.0907,0.085 ,0.0901,0.0908,0.0514,0.0922,0.1069]
14
0.0013



## 2.1.1 (7.5%) Clustering the Iris Classification problem - HAC

Load the Iris Dataset [Iris Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/iris.arff)

- Use single-link and complete link clustering algorithms
- State whether you normalize your data or not (your choice).  
- Show your results for clusterings using k = 2-7.  
- Graph the silhouette score for each k and discuss your results (i.e. what kind of clusters are being made).
---

In [50]:
# Iris Classification using single-link
!curl -s https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/iris.arff --output iris.arff
iris_data = arff.loadarff('iris.arff')
iris_df = pd.DataFrame(iris_data[0])
iris_df = iris_df.iloc[: , :-1]
iris_norm = preprocessing.normalize(iris_df, axis=0)

clf = HACClustering(k=5, link_type='single')
res = clf.fit(iris_norm)
res.score()

# Print clusters
res.print_clusters()

c:\Users\Alex\OneDrive\Desktop\Fall2021\CS472\CS472-Labs\.venv\lib\site-packages\numpy\lib\function_base.py:4454: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)


Num clusters: 5
SSE score: 0.1028


[0.0694,0.0911,0.0289,0.014 ]
49
0.0076

[0.0623,0.0609,0.0256,0.0173]
1
0.0000

[0.0864,0.0756,0.0959,0.0959]
97
0.0951

[0.0678,0.0662,0.0885,0.0978]
1
0.0000

[0.1079,0.1006,0.1289,0.1208]
2
0.0001



In [51]:
# Iris Classification using complete-link
clf = HACClustering(k=5, link_type='complete')
res = clf.fit(iris_norm)
res.score()

# Print clusters
res.print_clusters()

Num clusters: 5
SSE score: 0.0334


[0.0671,0.0854,0.028 ,0.0115]
33
0.0027

[0.0735,0.1003,0.0303,0.0189]
17
0.0022

[0.0818,0.0725,0.0845,0.0762]
52
0.0170

[0.0882,0.0762,0.104 ,0.106 ]
23
0.0043

[0.0952,0.0832,0.1146,0.1295]
25
0.0072



Discuss differences between single-link and complete-link

## 2.2.1 (7.5%) Clustering the Iris Classification problem: K-Means

Load the Iris Dataset [Iris Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/iris.arff)

Run K-Means on the Iris dataset using the output label as a feature and without using the output label as a feature

Requirements:
- State whether you normalize your data or not (your choice).  
- Show your results for clusterings using k = 2-7.  
- Graph the silhouette score for each k and discuss your results (i.e. what kind of clusters are being made).
---

In [ ]:
# Iris Classification without output label

In [ ]:
# Iris Classification with output label

Compare results and differences between using the output label and excluding the output label

## 2.2.2 (5%) Clustering the Iris Classification problem: K-Means

Requirements:
- Use the output label as an input feature
- Run K-Means 5 times with k=4, each time with different initial random centroids and discuss any variations in the results. 

In [ ]:
#K-Means 5 times

Discuss any variations in the results

## 3.1 (12.5%) Run the SK versions of HAC (both single and complete link) on iris including the output label and compare your results with those above.
Use the silhouette score for this iris problem(k = 2-7).  You may write your own code to do silhouette (optional extra credit) or you can use sklearn.metrics.silhouette_score. Please state if you coded your own silhouette score function to receive the extra credit points (described below). Discuss how helpful Silhouette appeared to be for selecting which clustering is best. You do not need to supply full Silhouette graphs, but you could if you wanted to.

Requirements
- Use the Sillhouette score for this iris problem (k= 2-7) 
- Use at least one other scoring function from [sklearn.metrics](https://scikit-learn.org/stable/modules/model_evaluation.html) and compare the results. State which metric was used. 
- Possible sklean metrics include (* metrics require ground truth labels):
    - adjusted_mutual_info_score*
    - adjusted_rand_score*
    - homogeneity_score*
    - completeness_score*
    - fowlkes_mallows_score*
    - calinski_harabasz_score
    - davies_bouldin_score
- Experiment using different hyper-parameters. Discuss Results

In [ ]:
# Load sklearn



*Record impressions*

## 3.2 (12.5%) Run the SK version of k-means on iris including the output label and compare your results with those above. 

Use the silhouette score for this iris problem(k = 2-7). You may write your own code to do silhouette (optional extra credit) or you can use sklearn.metrics.silhouette_score. Please state if you coded your own silhouette score function to receive the extra credit points (described below). Discuss how helpful Silhouette appeared to be for selecting which clustering is best. You do not need to supply full Silhouette graphs, but you could if you wanted to.

Requirements
- Use the Sillhouette score for this iris problem (k= 2-7) 
- Use at least one other scoring function form sklearn.metrics and compare the results. State which metric was used
- Experiment different hyper-parameters. Discuss Results

In [ ]:
# Load sklearn 



*Record impressions*

## 4. (Optional 5% extra credit) For your silhouette experiment above, write and use your own code to calculate the silhouette scores, rather than the SK or other version. 


*Show findings here*

In [ ]:
# Copy function Below